### Load corpus

In [49]:
import importlib
import LiB
importlib.reload(LiB)

'The input is a text file with each sentence separated by a newline and each token separated by a space.'
corpus = LiB.Corpus('corpus/br-text.txt', sentence_divider='\n', token_divider=' ')
# corpus = LiB.Corpus('zh_sent.txt', token_divider=' ', sentence_divider='\n', remove_token_divider=True)
# corpus = LiB.Corpus('ctb.txt', token_divider=' ', sentence_divider='\n', remove_token_divider=True)

'The input is a directory path with each file as an article, each sentence separated by a newline, and each token separated by a space.'
# corpus = LiB.Corpus('corpus', sentence_divider='\n', token_divider=' ')

'The input is a string with each sentence separated by a newline and each token separated by a space.'
# s = "you want to see the book\ncan you feed it to the doggie\nwhat's it\nget it"
# corpus = LiB.Corpus(s, sentence_divider='\n', token_divider=' ')

""" The corpus_ is a list of sentences, each sentence is a list of tokens.
If the token_divider is not None, the corpus_ should be a list of strings, each string is a sentence with tokens separated by the token_divider.
If the sentence_divider is not None, the corpus_ should be a string, with sentences separated by the sentence_divider. """;
# with open('sents_with_line_ends_without_repeat_en', 'rb') as f:
#     import pickle
#     corpus_ = pickle.load(f)
# corpus = LiB.Corpus(corpus_, token_divider=None)


In [213]:
import importlib
import LiB
importlib.reload(LiB)

'Init a model with parameters (explained in the code script)'
m = LiB.Model(corpus, in_rate=0.25, out_rate=0.01, life=3, update_rate=0.1, 
              inspect_context=True)

'RUN!!!'
# The model learns 200 steps
for step_id in range(0,201):
    # The model learns an article with 200 tokens in each step, and reports the metrics every 100 steps
    m.learn(step_id, article_length_max=200, report_interval=100);
m.clean_lexicon()
            
'Report'
print(f"Time cost for training: {sum(m.logs['time_cost'])//60} min {sum(m.logs['time_cost'])%60:.1f}s\n")

print('Orignal/LiB segmentation:')
m.segment_and_show()

0	  LexiconSize: 415  TypeLength: 4.2  TokenLength: 3.8	  EvalIndex: 0.431


100	  LexiconSize: 2015  TypeLength: 7.5  TokenLength: 8.7	  EvalIndex: 0.791
									Δ: 83.5%
200	  LexiconSize: 2338  TypeLength: 8.0  TokenLength: 9.2	  EvalIndex: 0.819
									Δ: 3.5%
Time cost for training: 0.0 min 3.4s

Orignal/LiB segmentation:
you |want |to |see |the |book 
you want to |see the book 

look |there's |a |boy |with |his |hat 
look there's |a boy |with |his |hat 

and |a |doggie 
and a |doggie 

you |want |to |look |at |this 
you want to |look at this 

look |at |this 
look at this 

have |a |drink 
have a |drink 

okay |now 
okay now 

what's |this 
what's this 

what's |that 
what's that 

what |is |it 
what is it 



### Misc

In [163]:
# m.save_lexicon('lexicon.txt', plain_text=False)
# m.load_lexicon('lexicon.txt', plain_text=False)

segmented_corpus = m.apply(m.corpus.test, flatten=False)
segmented_corpus

# m.save_segmented_corpus(path, segmented_corpus)
# codebook, encoded = m.encode_segmented_corpus(segmented_corpus)

[['you want to ', 'see the book '],
 ['look ', "there's a ", 'boy ', 'with ', 'his ', 'hat '],
 ['and a ', 'doggie '],
 ['you want to ', 'look at this '],
 ['look at this '],
 ['have a ', 'drink', ' '],
 ['okay now '],
 ["what's this "],
 ["what's that "],
 ['what is it ']]

In [18]:
import math

# The Huffman Coding algorithm comes from https://github.com/arnab132/Huffman-Coding-Python/blob/master/huffman.py
def huffman_coding(corpus, show_codebook=False):
    # Creating tree nodes
    class NodeTree(object):
        def __init__(self, left=None, right=None):
            self.left = left
            self.right = right

        def children(self):
            return (self.left, self.right)

        def nodes(self):
            return (self.left, self.right)

        def __str__(self):
            return '%s_%s' % (self.left, self.right)

    # Main function implementing huffman coding
    def huffman_code_tree(node, left=True, binString=''):
        if type(node) is str:
            return {node: binString}
        (l, r) = node.children()
        d = dict()
        d.update(huffman_code_tree(l, True, binString + '0'))
        d.update(huffman_code_tree(r, False, binString + '1'))
        return d

    # Calculating frequency
    freq = {}
    for c in corpus:
        if c in freq:
            freq[c] += 1
        else:
            freq[c] = 1

    freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)

    nodes = freq

    while len(nodes) > 1:
        (key1, c1) = nodes[-1]
        (key2, c2) = nodes[-2]
        nodes = nodes[:-2]
        node = NodeTree(key1, key2)
        nodes.append((node, c1 + c2))

        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)

    huffman_code = huffman_code_tree(nodes[0][0])
    if show_codebook:
        print("[Huffman Encoding]:")
        for token, code in huffman_code.items():
            print(f"{token}:\t{code}")

    return huffman_code, nodes[0][0]

def calculate_MDL(segmented_corpus, tree, encoding, show_encoded_corpus=False):
    # serialize the Huffman tree to a binary string with the labels of the leafs of the tree, by depth-first search
    def serialize_dfs(node, label='', tree='', values=[]):
        tree += label
        if type(node) is str:
            tree += '#'
            values.append(node)
            return tree, values
        else:
            (l, r) = node.children()
            tree, values = serialize_dfs(l, '0', tree, values)
            tree, values = serialize_dfs(r, '1', tree, values)
        return tree, values

    serialized_tree, serialized_values = serialize_dfs(tree) # serialize the tree to a binary string ('0/1') with the label of its leaves ('#'), and a list of to store the values of the leaves.
    # the number of bits to store the tree structure
    tree_bits = math.ceil(math.log2(3**len(serialized_tree))) # the string is trinary (0/1/#), it would be converted to binary to calculate the number of bits
    # the number of bits to store the values of the tree leaves (the dictionary)
    values_bits = len('\t'.join(serialized_values).encode('utf-8')) * 8 # the list would be represented by a string that is separated by '\t', and then encoded to utf-8 to calculate the number of bits
    model_bits = tree_bits + values_bits

    encoded_corpus = ""
    for token in segmented_corpus:
        encoded_corpus += encoding[token]
    encoded_bits = len(encoded_corpus) # the number of bits to store the encoded corpus
    if show_encoded_corpus:
        print("[Encoded Corpus]:")
        print(encoded_corpus)

    mdl = model_bits + encoded_bits
    return model_bits, encoded_bits, mdl

flat_segmented_corpus = m.apply(m.corpus.train, flatten=True)
huffman_code, huffman_tree = huffman_coding(flat_segmented_corpus, show_codebook=True)
dictionary_bits, encoded_bits, mdl = calculate_MDL(flat_segmented_corpus, 
                                                   huffman_tree, huffman_code, 
                                                   show_encoded_corpus=False)
print(f'Codebook:\t{dictionary_bits} bits\nEncoded_corpus:\t{encoded_bits} bits\nMDL:\t\t{mdl} bits')

[Huffman Encoding]:
 :	00000000
p:	000000010
you want to :	0000000110
know what :	00000001110
drink:	00000001111
s :	00000010
your :	00000011
v:	000001000
at :	000001001
how many :	000001010
pa:	000001011
get :	000001100
gonna :	000001101
some :	000001110
y :	000001111
mommy's ring :	00001000000
scratchy face :	00001000001
what color :	00001000010
blue :	00001000011
your s:	00001000100
in here :	00001000101
daddy's scratchy face :	00001000110
comes :	00001000111
marie :	0000100100
ch :	0000100101
us :	00001001100
you have :	00001001101
i have :	00001001110
lift :	00001001111
rt :	00001010000
it's not a :	00001010001
can i :	00001010010
mic:	00001010011
fun :	00001010100
in the mirror :	00001010101
keys :	00001010110
the brush :	00001010111
asleep :	00001011000
will you :	00001011001
like your :	00001011010
read a book :	00001011011
numbers :	00001011100
don't touch :	00001011101
having :	00001011110
six :	00001011111
nose :	0000110000
shoe :	0000110001
spoon :	0000110010
apple :	000011

In [17]:
'validate the serializer'
'by comparing the serialized tree with the serialized-deserialized-serialized tree'
# serialize the tree to a string by depth-first search
def serialize_dfs(node, label='', tree='', values=None):
    if values is None:
        values = []

    tree += label
    if type(node) is str:
        tree += '#'
        values.append(node)
        return tree, values
    else:
        (l, r) = node.children()
        tree, values = serialize_dfs(l, '0', tree, values)
        tree, values = serialize_dfs(r, '1', tree, values)
    return tree, values

# deserialize the tree from a string by depth-first search
def deserialize_dfs(tree, values):
    # Creating tree nodes
    class NodeTree(object):
        def __init__(self, left=None, right=None):
            self.left = left
            self.right = right

        def children(self):
            return (self.left, self.right)

        def nodes(self):
            return (self.left, self.right)

        def __str__(self):
            return '%s_%s' % (self.left, self.right)

    def f(tree, values):
        if tree[0] == '#':
            node, tree, values = values[0], tree[1:], values[1:]
        else:
            node = NodeTree()
            node.left, tree, values = f(tree[1:], values)
            node.right, tree, values = f(tree[1:], values)
        return node, tree, values

    while True:
        node, tree, values = f(tree, values)
        if len(values) == 0:
            break
    return node

# Calculating frequency
freq = {}
for c in flat_segmented_corpus:
    if c in freq:
        freq[c] += 1
    else:
        freq[c] = 1

freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)

nodes = freq

while len(nodes) > 1:
    (key1, c1) = nodes[-1]
    (key2, c2) = nodes[-2]
    nodes = nodes[:-2]
    node = NodeTree(key1, key2)
    nodes.append((node, c1 + c2))
    nodes = sorted(nodes, key=lambda x: x[1], reverse=True)

serialized_tree, serialized_values = serialize_dfs(nodes[0][0])

new_nodes = deserialize_dfs(serialized_tree, serialized_values)
new_serialized_tree, new_serialized_values = serialize_dfs(new_nodes)

print(serialized_tree)
print(new_serialized_tree)

00000000#10#10#10#1#10#1#1000#1#10#1#100#1#10#1#1000000#1#10#1#100#1#10#1#100#1#100#1#10#1#10000#1#10#1#100#1#10#1#1000#1#10#1#100#1#10#1#10000#1#10#1#100#1#10#1#1000#1#10#1#100#1#10#1#10000000#1#10#1#1#1000#1#10#1#100#1#10#1#100#1#10#1#100000#1#10#1#100#1#10#1#1000#1#10#1#100#1#10#1#10000#1#10#1#100#1#10#1#1000#1#10#1#100#1#10#1#100000#1#10#1#1#1000#1#10#1#100#1#10#1#10#10#10#1#10000#10#10#1#1000#1#1#10#1#100000#1#10#1#100#1#10#1#1000#1#1#100#1#10#1#1000#1#10#1#1000#1#10#1#10#10#10#1#100#100#10#1#10#1#10000#10#1#1#10#10#1#100000#1#10#1#100#1#10#1#1000#1#10#1#100#1#10#1#100#1#1000#1#10#1#1#10000000000#1#10#1#100#1#10#1#1000#1#10#1#100#1#10#1#10000#1#10#1#100#1#10#1#1000#1#10#1#100#1#10#1#100000#1#10#1#100#1#10#1#1000#1#10#1#100#1#10#1#10000#1#10#1#100#1#10#1#1000#1#10#1#100#1#10#1#10000#1#10#1#100#1#10#1#100000#1#10#1#1#1000#1#10#1#100#1#10#1#100#1#10#1#1000000#1#10#1#100#1#10#1#1000#1#10#1#100#1#10#1#10000#1#10#1#100#1#10#1#1000#1#10#1#100#1#10#1#1000000#1#10#1#100#1#10#1#1000#1#10#1#